In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import logging
import penaltyblog as pb
from types import SimpleNamespace
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
import sys
sys.path.append('../src')
from dixon_coles import create_team_ratings, simulate_league, process_simulation_results
from scraping_utils import initialize_webdriver, scroll_page, get_results, get_fixtures, get_table

In [ ]:
# Set up basic logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Functions

These are now imported from src.scrapeing_utils

In [ ]:
def initialize_webdriver(url):

    # initialize webdriver
    driver = webdriver.Chrome()
    # load the website
    driver.get(url)

    return driver

In [ ]:
def scroll_page(driver):

    # Scroll down repeatedly (to load more content)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for loading

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # No more content to load
        last_height = new_height

In [ ]:
def get_results(driver):

    # Get teams
    elements = driver.find_elements(By.CLASS_NAME, 'teamname')
    teams = [elem.text for elem in elements]

    # Get goals
    elements = driver.find_elements(By.CLASS_NAME, 'scorefs')
    goals = [elem.text for elem in elements]

    # Check if the number of teams and goals match and only use games where the result is known
    teams = teams[-len(goals):]

    # create dataframe
    df = pd.DataFrame(columns=['team_home', 'team_away', 'goals_home', 'goals_away'])
    df.team_home = teams[0::2]
    df.team_away = teams[1::2]
    df.goals_home = goals[0::2]
    df.goals_away = goals[1::2]

    return df

In [ ]:
def get_fixtures(driver):
    
    # get teams
    elements = driver.find_elements(By.CLASS_NAME, 'teamname')
    teams = [elem.text for elem in elements]

    # re-create fixtures
    fixtures = pd.DataFrame(columns=['Home', 'Away'])
    fixtures.Home = teams[0::2]
    fixtures.Away = teams[1::2]
    
    return fixtures

In [ ]:
def get_table(driver):

    # get teams
    elements = driver.find_elements(By.CLASS_NAME, 'text-start')
    teams = [elem.text for elem in elements]
    teams = teams[7::5]

    # get stats
    elements = driver.find_elements(By.CLASS_NAME, 'text-center')
    stats = [elem.text for elem in elements]
    stats = stats[6::]

    table = pd.DataFrame(columns=['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts'])
    table.Rk = np.arange(1, len(teams)+1)
    table.Squad = teams
    table.MP = stats[0::6]
    table.W = stats[1::6]
    table.D = stats[2::6]
    table.L = stats[3::6]
    table.GF = [x.split('–')[0] for x in stats[4::6]]
    table.GA = [x.split('–')[1] for x in stats[4::6]]
    table.GD = table['GF'].astype(int) - table['GA'].astype(int)
    table.Pts = stats[5::6]

    # convert to numeric
    table[['Rk', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts']] = (
        table[['Rk', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts']].apply(pd.to_numeric, errors='coerce')
    )

    return table

# Scrape data

In [ ]:
#my_group = 3
#driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M7!etejp25/group/{my_group}/results')
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M7!etejp25/results') # all results
scroll_page(driver)

In [ ]:
df = get_results(driver)
#df.to_csv(f'../data/results_kevat25.csv', index=False)
df

In [ ]:
my_group = 4
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M7!etejp25/group/{my_group}/fixtures')
scroll_page(driver)

In [ ]:
fixtures = get_fixtures(driver)
#fixtures.to_csv(f'../data/fixtures_kevat25.csv', index=False)
fixtures

In [ ]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M7!etejp25/group/{my_group}/')

In [ ]:
table = get_table(driver)
#table.to_csv(f'../data/table_syksy1_25.csv', index=False)
table

# Model

In [ ]:
clf = pb.models.DixonColesGoalModel(
    df["goals_home"], df["goals_away"], df["team_home"], df["team_away"],
)

clf.fit()

In [ ]:
clf.get_params()

# Ratings

In [ ]:
teams = df['team_home'].unique()
args = SimpleNamespace(save_simulation_results=True, league=f'Syksy')
ratings_df = create_team_ratings(clf, teams, args, export_ratings=True)

# Simulation

In [ ]:
nr_simulations = 1000
simulation_results_df = simulate_league(clf, table, fixtures, nr_simulations)

In [ ]:
args = SimpleNamespace(save_simulation_results=True, league='Syksy I')
process_simulation_results(simulation_results_df, nr_simulations, args)

In [ ]:
clf.predict('Trikiinit', 'PK-35/Äijät')

# Kevät analysis

This does not seem to work...

### Group 1

In [ ]:
clf_1 = pb.models.DixonColesGoalModel(
    df["goals_home"], df["goals_away"], df["team_home"], df["team_away"],
)

clf_1.fit()

display(clf_1.get_params())

In [ ]:
clf.get_params()

In [ ]:
import plotly.graph_objects as go   

In [ ]:
teams = df.team_home.unique()

In [ ]:
string = 'attack_Cosmos'
string[7:]

In [ ]:
# Extract values
attack_values = [value for key, value in clf_1.get_params().items() if key.startswith('attack')]
defence_values = [value for key, value in clf_1.get_params().items() if key.startswith('defence')]

# Create a combined figure manually
fig = go.Figure()

# Add attack histogram
fig.add_trace(go.Histogram(
    x=attack_values,
    name='Attack',
    opacity=0.6,
    marker_color='blue'
))

# Add defence histogram
fig.add_trace(go.Histogram(
    x=defence_values,
    name='Defence',
    opacity=0.6,
    marker_color='red'
))

# Update layout
fig.update_layout(
    barmode='overlay',  # or 'group' if you want them side by side
    title='Attack vs Defence Parameter Distributions',
    xaxis_title='Value',
    yaxis_title='Count',
    legend_title='Parameter Type'
)

fig.show()


In [ ]:
# Extract values
attack_values = [value for key, value in clf.get_params().items() if ((key[7:] in teams) & (key.startswith('attack_')))]
defence_values = [value for key, value in clf.get_params().items() if ((key[8:] in teams) & (key.startswith('defence_')))]

# Create a combined figure manually
fig = go.Figure()

# Add attack histogram
fig.add_trace(go.Histogram(
    x=attack_values,
    name='Attack',
    opacity=0.6,
    marker_color='blue'
))

# Add defence histogram
fig.add_trace(go.Histogram(
    x=defence_values,
    name='Defence',
    opacity=0.6,
    marker_color='red'
))

# Update layout
fig.update_layout(
    barmode='overlay',  # or 'group' if you want them side by side
    title='Attack vs Defence Parameter Distributions',
    xaxis_title='Value',
    yaxis_title='Count',
    legend_title='Parameter Type'
)

fig.show()


In [ ]:
attack_values = [value for key, value in clf.get_params().items() if ((key[7:] in teams) & (key.startswith('attack_')))]
defence_values = [value for key, value in clf.get_params().items() if ((key[8:] in teams) & (key.startswith('defence_')))]

attack_values_1 = [value for key, value in clf_1.get_params().items() if key.startswith('attack')]
defence_values_1 = [value for key, value in clf_1.get_params().items() if key.startswith('defence')]

team_keys = [key[7:] for key, value in clf.get_params().items() if ((key[7:] in teams) & (key.startswith('attack_')))]
team_keys_1 = [key[7:] for key, value in clf_1.get_params().items() if ((key.startswith('attack_')))]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=attack_values,
    y=defence_values,
    opacity=0.6,
    marker_color='blue',
    mode='markers',
    hovertext=team_keys,
    name='clf'
))

fig.add_trace(go.Scatter(
    x=attack_values_1,
    y=defence_values_1,
    opacity=0.6,
    marker_color='red',
    mode='markers',
    hovertext=team_keys_1,
    name='clf_1'
))

In [ ]:
display(clf.get_params()['rho'])
display(clf_1.get_params()['rho'])

In [ ]:
display(clf.get_params()['home_advantage'])
display(clf_1.get_params()['home_advantage'])

In [ ]:
display(np.mean(attack_values))
display(np.mean(attack_values_1))

In [ ]:
attack_values_clf = [value for key, value in clf.get_params().items() if ((key.startswith('attack_')))]
display(np.mean(attack_values_clf))

### Group 2

In [ ]:
clf_2 = pb.models.DixonColesGoalModel(
    df["goals_home"], df["goals_away"], df["team_home"], df["team_away"],
)

clf_2.fit()

display(clf_2.get_params())

### Group 3

In [ ]:
clf_3 = pb.models.DixonColesGoalModel(
    df["goals_home"], df["goals_away"], df["team_home"], df["team_away"],
)

clf_3.fit()

display(clf_3.get_params())

### Group comparison

In [ ]:
attack_values_list = []
defence_values_list = []
for my_clf in [clf_1, clf_2, clf_3]:
    attack_values_list.append([value for key, value in my_clf.get_params().items() if key.startswith('attack')])
    defence_values_list.append([value for key, value in my_clf.get_params().items() if key.startswith('defence')])

In [ ]:
print('Individual group models')
for i in range(3):
    group_mean_attack = np.mean(attack_values_list[i])
    group_mean_defence = np.mean(defence_values_list[i])
    print(f'Group {i+1} mean attack: {group_mean_attack}, mean defence: {group_mean_defence}')

In [ ]:
team_list = []
for my_clf in [clf_1, clf_2, clf_3]:
    team_list.append([key[7:] for key, value in my_clf.get_params().items() if key.startswith('attack')])

In [ ]:
attack_values_list = []
defence_values_list = []
for i in range(3):
    attack_values_list.append([value for key, value in clf.get_params().items() if ((key.startswith('attack')) & (key[7:] in team_list[i]))])
    defence_values_list.append([value for key, value in clf.get_params().items() if ((key.startswith('defence')) & (key[8:] in team_list[i]))])

In [ ]:
print('Total model')
for i in range(3):
    group_mean_attack = np.mean(attack_values_list[i])
    group_mean_defence = np.mean(defence_values_list[i])
    print(f'Group {i+1} mean attack: {group_mean_attack}, mean defence: {group_mean_defence}')

In [ ]:
clf_1.predict('Trikiinit', 'Kullervo')

In [ ]:
clf.predict('Trikiinit', 'Kullervo')

In [ ]:
clf_2.predict('Valtti/TDJ', 'FC Lähiö')

In [ ]:
clf.predict('Valtti/TDJ', 'FC Lähiö')

In [ ]:
clf_3.predict('Valtti/4', 'LPS/7')

In [ ]:
clf.predict('Valtti/4', 'LPS/7')

## Combined model

In [ ]:
display(clf.get_params())

In [ ]:
keys_to_exclude = {'home_advantage', 'rho'}
aux_dict = {
    **{k: v for k, v in clf_1.get_params().items() if k not in keys_to_exclude},
    **{k: v for k, v in clf_2.get_params().items() if k not in keys_to_exclude},
    **{k: v for k, v in clf_3.get_params().items() if k not in keys_to_exclude},
}

# Sort the dictionary by keys
aux_dict_sorted = {key: aux_dict[key] for key in sorted(aux_dict)}

aux_dict_sorted

In [ ]:
np.array(list(aux_dict_sorted.values()))

In [ ]:
clf._params[:-2] = np.array(list(aux_dict_sorted.values()))

In [ ]:
clf.get_params()

In [ ]:
clf.__dict__['teams_home']

In [ ]:
teams = fixtures.Home.unique()
args = SimpleNamespace(save_simulation_results=False, league=f'Kevät')
ratings_df = create_team_ratings(clf, teams, args, export_ratings=False)

# Misc

In [ ]:
all_ratings = []

In [ ]:
all_ratings.append(ratings_df)

In [ ]:
all_ratings_df = pd.concat(all_ratings).sort_values(by='goal_difference_rating', ascending=False).reset_index(drop=True)
all_ratings_df

In [ ]:
all_ratings_df.to_csv('../output/ratings_Kevät.csv', index=False)

In [ ]:
import dataframe_image as dfi
dfi.export(all_ratings_df, f"../output/ratings_Kevät.png", table_conversion='matplotlib',)

# Development

## results

In [ ]:
# Initialize the Chrome driver
driver = webdriver.Chrome()

# Load the website
driver.get('https://tulospalvelu.palloliitto.fi/category/M7!etejp25/group/1/results')

In [ ]:
elements = driver.find_elements(By.CLASS_NAME, 'teamname')
teams = [elem.text for elem in elements]
print(teams)

In [ ]:
elements = driver.find_elements(By.CLASS_NAME, 'scorefs')
goals = [elem.text for elem in elements]
print(goals)

In [ ]:
elements = driver.find_elements(By.CLASS_NAME, 'match_status')
status = [elem.text for elem in elements]
print(status)

In [ ]:
df = pd.DataFrame(columns=['team_home', 'team_away', 'goals_home', 'goals_away'])
df

In [ ]:
df.team_home = teams[0::2]
df.team_away = teams[1::2]
df.goals_home = goals[0::2]
df.goals_away = goals[1::2]
df

## fixtures

In [ ]:
# Initialize the Chrome driver
driver = webdriver.Chrome()

# Load the website
driver.get('https://tulospalvelu.palloliitto.fi/category/M7!etejp25/group/1/fixtures')

In [ ]:
elements = driver.find_elements(By.CLASS_NAME, 'teamname')
teams = [elem.text for elem in elements]
print(teams)

In [ ]:
fixtures = pd.DataFrame(columns=['Home', 'Away'])
fixtures.Home = teams[0::2]
fixtures.Away = teams[1::2]
fixtures

## table

In [ ]:
# Initialize the Chrome driver
driver = webdriver.Chrome()

# Load the website
driver.get('https://tulospalvelu.palloliitto.fi/category/M7!etejp25/group/1/')

In [ ]:
elements = driver.find_elements(By.CLASS_NAME, 'text-start')
teams = [elem.text for elem in elements]
teams = teams[7::5]
print(teams)

In [ ]:
elements = driver.find_elements(By.CLASS_NAME, 'text-center')
stats = [elem.text for elem in elements]
stats = stats[6::]
print(stats)

In [ ]:
table = pd.DataFrame(columns=['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts'])
table

In [ ]:
stats[4::6][0].split('–')[0]

In [ ]:
[x.split('–')[0] for x in stats[4::6]]

In [ ]:
table.Rk = np.arange(1, len(teams)+1)
table.Squad = teams
table.MP = stats[0::6]
table.W = stats[1::6]
table.D = stats[2::6]
table.L = stats[3::6]
table.GF = [x.split('–')[0] for x in stats[4::6]]
table.GA = [x.split('–')[1] for x in stats[4::6]]
table.GD = table['GF'].astype(int) - table['GA'].astype(int)
table.Pts = stats[5::6]
table

## lineups

### Get match ids

In [ ]:
# Initialize the Chrome driver
driver = webdriver.Chrome()

# Load the website
driver.get('https://tulospalvelu.palloliitto.fi/category/M7!etejp25/results')

In [ ]:
# Scroll down repeatedly (to load more content)

last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Wait for loading

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # No more content to load
    last_height = new_height

In [ ]:
# Find all elements with a 'matchid' attribute
elements = driver.find_elements(By.XPATH, '//*[@matchid]')
# Extract all match IDs
match_ids = [el.get_attribute("matchid") for el in elements]
print(len(np.unique(match_ids)))

### Get lineups

In [ ]:
# Initialize the Chrome driver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)  # wait up to 10 seconds

all_game_data = []
for match_id in np.unique(match_ids):
    # Load the website
    driver.get(f'https://tulospalvelu.palloliitto.fi/match/{match_id}/lineups')
    wait.until(EC.presence_of_element_located((By.ID, "matchscore")))
    
    # Extract team names
    home_team = driver.find_element(By.CSS_SELECTOR, "#A_team .teamname").text.strip()
    away_team = driver.find_element(By.CSS_SELECTOR, "#B_team .teamname").text.strip()

    # Extract match score (e.g., "2 – 1")
    score = driver.find_element(By.ID, "matchscore").text.strip().split("\n")[0]  # Remove subscore like (0–0)

    home_team_goals = int(score.split('–')[0])
    away_team_goals = int(score.split('–')[1])

    # Print result
    print(f"{home_team} vs {away_team}: {home_team_goals} – {away_team_goals}")

    # Find both player tables
    player_tables = driver.find_elements(By.XPATH, '//div[contains(@class, "playerlist")]')
    
    all_players = []
    teams = [home_team, away_team]
    goals = [home_team_goals, away_team_goals]
    # Iterate through each player table (0 = first team, 1 = second team)
    for idx, table in enumerate(player_tables):
        team_label = teams[idx]
        
        # Find all player rows in this table
        rows = table.find_elements(By.XPATH, './/table//tr')
        
        for row in rows:
            try:
                # Extract shirt number
                shirt_number = row.find_element(By.XPATH, './/span[contains(@class, "shirtnumber")]').text.strip()
            except:
                shirt_number = ""

            try:
                # Extract name (text from the namenarrow span)
                name_elem = row.find_element(By.XPATH, './/span[contains(@class, "undefined")]')
                name_text = name_elem.text.strip()

                # Remove "| MV", "| C", etc. if present
                player_name = name_text.split('|')[0].strip()
            except:
                player_name = "Unknown"

            all_players.append({
                "team": team_label,
                "shirt_number": shirt_number,
                "name": player_name,
                "goals_scored": goals[idx],
                "goals_conceded": goals[1 - idx]  # Goals conceded is the other team's score
            })

    # Print all players
    for player in all_players:
        print(f"{player['team']} - {player['shirt_number']} {player['name']}: Team goals scored: {player['goals_scored']}, Goals conceded: {player['goals_conceded']}")

    # Append to all game data
    all_game_data.append({
        "match_id": match_id,
        "home_team": home_team,
        "away_team": away_team,
        "home_goals": home_team_goals,
        "away_goals": away_team_goals,
        "players": all_players
    })

# Close the driver after scraping
driver.quit()

In [ ]:
len(all_game_data)

In [ ]:
df = pd.DataFrame(all_game_data)
df

In [ ]:
df_exploded = df.explode('players').reset_index(drop=True)
df_exploded

In [ ]:
# Normalize the dictionary into columns
df_normalized = pd.concat(
    [df_exploded.drop(columns=['players']),
     pd.json_normalize(df_exploded['players'])],
    axis=1
)
df_normalized

In [ ]:
df_normalized.to_csv('../data/player_data_kevat.csv', index=False)